In [22]:
import numpy as np
import torch
from torch import nn
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

training_data = pd.read_csv("../gym-unbalanced-disk/disc-benchmark-files/training-val-test-data.csv", header=None)

np.random.seed(42)

ulist = training_data.iloc[:, 0].tolist()[1:]   # Input voltage
ylist = training_data.iloc[:, 1].tolist()[1:]  # Output angle

# Convert elements to floats
ulist = [float(val) for val in ulist]
ylist = [float(val) for val in ylist]

# Normalize the data
scaler_u = MinMaxScaler()
scaler_y = MinMaxScaler()

ulist = np.array(ulist).reshape(-1, 1)
ylist = np.array(ylist).reshape(-1, 1)

ulist_scaled = scaler_u.fit_transform(ulist)
ylist_scaled = scaler_y.fit_transform(ylist)

def make_OE_data(udata, ydata, nf=100):
    U = [] 
    Y = [] 
    for k in range(nf, len(udata) + 1):
        U.append(udata[k - nf:k]) #a)
        Y.append(ydata[k - nf:k]) #a)
    return np.array(U), np.array(Y)

nfuture = 50
U_LSTM, Y_LSTM = make_OE_data(ulist_scaled.flatten(), ylist_scaled.flatten(), nf=nfuture)

U_LSTM = torch.tensor(U_LSTM, dtype=torch.float64).unsqueeze(-1).clone().detach()  # shape (batch_size, sequence_length, input_size)
Y_LSTM = torch.tensor(Y_LSTM, dtype=torch.float64).clone().detach()

split_ratio = 0.8
split_index = int(split_ratio * len(U_LSTM))

Utrain, Ytrain_LSTM = U_LSTM[:split_index], Y_LSTM[:split_index]
Uval, Yval_LSTM = U_LSTM[split_index:], Y_LSTM[split_index:]


In [24]:
# 3 Layers LSTM
class LSTM(nn.Module):
    def __init__(self, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.hidden_size = hidden_size
        self.hidden_size_2 = int(2*hidden_size)

        self.input_size = 1
        self.output_size = 1
        
        net = lambda n_in, n_out: nn.Sequential( 
            nn.Linear(n_in, 128),  
            nn.ReLU(),  
            nn.Linear(128, 64),  
            nn.ReLU(), 
            nn.Linear(64, n_out)   
        )
        
        self.lstm1 = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size_2, batch_first=True).double()
        self.lstm2 = nn.LSTM(input_size=self.hidden_size_2, hidden_size=self.hidden_size_2, batch_first=True).double()
        self.lstm3 = nn.LSTM(input_size=self.hidden_size_2, hidden_size=hidden_size, batch_first=True).double()
        
        self.h2o = net(hidden_size + self.input_size, self.output_size).double() 

    def forward(self, inputs):
        hiddens1, (h_n1, c_n1) = self.lstm1(inputs)
        hiddens2, (h_n2, c_n2) = self.lstm2(hiddens1)
        hiddens3, (h_n3, c_n3) = self.lstm3(hiddens2)
        
        combined = torch.cat((hiddens3, inputs), dim=2)
        h2o_input = combined.view(-1, self.hidden_size + self.input_size)
        y_predict = self.h2o(h2o_input).view(inputs.shape[0], inputs.shape[1])
        
        return y_predict

# 4 Layers LSTM
# class LSTM(nn.Module):
#     def __init__(self, hidden_size):
#         super(LSTM, self).__init__()
#         self.hidden_size = hidden_size
#         self.hidden_size_2 = int(2*hidden_size)
#         self.hidden_size_3 = int(4*hidden_size)

#         self.input_size = 1
#         self.output_size = 1
        
#         net = lambda n_in, n_out: nn.Sequential( 
#             nn.Linear(n_in, 128),  # Add a hidden layer
#             nn.ReLU(),  # Activation function
#             nn.Linear(128, 64),  # Another hidden layer
#             nn.ReLU(),  # Activation function
#             nn.Linear(64, n_out)   # Output layer
#         )
        
#         self.lstm1 = nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size_2, batch_first=True).double()
#         self.lstm2 = nn.LSTM(input_size=self.hidden_size_2, hidden_size=self.hidden_size_3, batch_first=True).double()
#         self.lstm3 = nn.LSTM(input_size=self.hidden_size_3, hidden_size=self.hidden_size_2, batch_first=True).double()
#         self.lstm4 = nn.LSTM(input_size=self.hidden_size_2, hidden_size=hidden_size, batch_first=True).double()
        
#         self.h2o = net(hidden_size + self.input_size, self.output_size).double() 

#     def forward(self, inputs):
#         hiddens1, (h_n1, c_n1) = self.lstm1(inputs)
#         hiddens2, (h_n2, c_n2) = self.lstm2(hiddens1)
#         hiddens3, (h_n3, c_n3) = self.lstm3(hiddens2)
#         hiddens4, (h_n3, c_n3) = self.lstm4(hiddens3)
        
        
#         combined = torch.cat((hiddens4, inputs), dim=2)
#         h2o_input = combined.view(-1, self.hidden_size + self.input_size)
#         y_predict = self.h2o(h2o_input).view(inputs.shape[0], inputs.shape[1])
        
#         return y_predict

n_hidden_nodes = 64
n_burn = 10
epochs = 100
batch_size = 32  
lr = 0.001

model_name= f"LSTM_2layers_{nfuture}nfuture_{n_burn}n_burn_{lr}lr.pth"

In [7]:
model = LSTM(n_hidden_nodes)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.1, verbose=True)

train_losses = []
val_losses = []
best_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    for i in range(0, len(Utrain), batch_size):
        Uin = Utrain[i:i + batch_size]
        Y_real = Ytrain_LSTM[i:i + batch_size]
        
        Y_predict = model(Uin)

        residual = Y_real - Y_predict
        Loss = torch.mean(residual[:, n_burn:]**2)
        
        optimizer.zero_grad()
        Loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        Loss_val = torch.mean((model(Uval)[:, n_burn:] - Yval_LSTM[:, n_burn:])**2)**0.5
        Loss_train = torch.mean((model(Utrain)[:, n_burn:] - Ytrain_LSTM[:, n_burn:])**2)**0.5
        print(f'epoch={epoch}, Validation NRMS={Loss_val.item():.2%}, Train NRMS={Loss_train.item():.2%}')
        scheduler.step(Loss_val)
        val_losses.append(Loss_val.item())
        train_losses.append(Loss_train.item())

        if Loss_val < best_val_loss:
            best_val_loss = Loss_val
            #torch.save(model.state_dict(), model_name)


torch.save(model.state_dict(), model_name)

import matplotlib.pyplot as plt
# Plot the losses
plt.plot(range(epochs), [loss for loss in train_losses], label='Training Loss')
plt.plot(range(epochs), [loss for loss in val_losses], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (%)')
plt.title('Training and Validation Losses')
plt.legend()
plt.show()



In [26]:
import numpy as np
import torch
from torch import nn
from sklearn.preprocessing import MinMaxScaler


# Load training data
out = np.load('../gym-unbalanced-disk/disc-benchmark-files/training-val-test-data.npz')
u_train = out['u']  # Input voltage
th_train = out['th']  # Output angle

scaler_u = MinMaxScaler()
scaler_y = MinMaxScaler()

u_train = np.array(u_train).reshape(-1, 1)
th_train = np.array(th_train).reshape(-1, 1)

u_train = scaler_u.fit_transform(u_train)
th_train = scaler_y.fit_transform(th_train)


nfuture = 50 
U_LSTM, Y_LSTM = make_OE_data(u_train, th_train, nf=nfuture)


U_LSTM = torch.tensor(U_LSTM, dtype=torch.float64)
Y_LSTM = torch.tensor(Y_LSTM, dtype=torch.float64)

reg = LSTM(n_hidden_nodes)
reg.load_state_dict(torch.load(model_name))
reg.eval()

with torch.no_grad():
    Ytrain_pred = reg(U_LSTM)

Ytrain_pred = Ytrain_pred.detach().numpy().flatten()
Y_LSTM = Y_LSTM.numpy().flatten()

print('train prediction errors:')
print('RMS:', np.mean((Ytrain_pred - Y_LSTM)**2)**0.5, 'radians')
print('RMS:', np.mean((Ytrain_pred - Y_LSTM)**2)**0.5 / (2 * np.pi) * 360, 'degrees')
print('NRMS:', np.mean((Ytrain_pred - Y_LSTM)**2)**0.5 / Y_LSTM.std() * 100, '%')


data = np.load('../gym-unbalanced-disk/disc-benchmark-files/hidden-test-prediction-submission-file.npz')
upast_test = data['upast']  # N by u[k-15],u[k-14],...,u[k-1]
thpast_test = data['thpast']  # N by y[k-15],y[k-14],...,y[k-1]

Xtest = np.concatenate([upast_test[:, 1:], thpast_test], axis=1)
Xtest = torch.tensor(Xtest, dtype=torch.float64)

Ypredict = reg(Xtest)
Ypredict = Ypredict.detach().numpy().flatten()

# Save predictions
np.savez('../gym-unbalanced-disk/disc-benchmark-files/hidden-test-prediction-example-submission-file.npz', upast=upast_test, thpast=thpast_test, thnow=Ypredict)



train prediction errors:
RMS: 0.07485682194341 radians
RMS: 4.288979965119682 degrees
NRMS: 63.69137266580939 %


RuntimeError: mat1 and mat2 shapes cannot be multiplied (645x29 and 1x512)

In [16]:
# Simulation
out = np.load('../gym-unbalanced-disk/disc-benchmark-files/training-val-test-data.npz')
th_train = out['th'] #th[0],th[1],th[2],th[3],...
u_train = out['u'] #u[0],u[1],u[2],u[3],...

data = np.load('../gym-unbalanced-disk/disc-benchmark-files/hidden-test-simulation-submission-file.npz')
u_test = data['u']
th_test = data['th'] #only the first 50 values are filled the rest are zeros


nfuture = 50
convert = lambda x: [torch.tensor(xi,dtype=torch.float64) for xi in x]
U_LSTM, Y_LSTM = convert(make_OE_data(u_train, th_train, nf=nfuture))

reg = LSTM(n_hidden_nodes) 
reg.load_state_dict(torch.load(model_name))
reg.eval()

# Simulation
out = np.load('../gym-unbalanced-disk/disc-benchmark-files/training-val-test-data.npz')
th_train = out['th']
u_train = out['u']

data = np.load('../gym-unbalanced-disk/disc-benchmark-files/hidden-test-simulation-submission-file.npz')
u_test = data['u']
th_test = data['th']
nfuture = 50
convert = lambda x: [torch.tensor(xi, dtype=torch.float64) for xi in x]
U_LSTM, Y_LSTM = convert(make_OE_data(u_train, th_train, nf=nfuture))

def fmodel_OE(upast):
    x = torch.as_tensor(upast, dtype=torch.float64).unsqueeze(0).unsqueeze(-1)  
    with torch.no_grad():
        ypred = reg(x)  
    return ypred.numpy().flatten()[-1]  

# OE simulation function
def OE_Simulation(U_data, f, nf, skip):
    U_past = U_data[:skip]  
    U_past = U_past.tolist() 
    Y_sim = U_data[:skip].tolist() 

    for i in range(skip, len(U_data)):
        y_current = f(U_past[-nf:])  
        Y_sim.append(y_current)  
        U_past.append(U_data[i])

    return np.array(Y_sim)

skip = 50

# OE simulation on training data
th_train_sim = OE_Simulation(u_train, fmodel_OE, 50, skip=skip)

print('train simulation errors:')
print('RMS:', np.mean((th_train_sim[skip:] - th_train[skip:])**2)**0.5, 'radians')
print('RMS:', np.mean((th_train_sim[skip:] - th_train[skip:])**2)**0.5 / (2 * np.pi) * 360, 'degrees')
print('NRMS:', np.mean((th_train_sim[skip:] - th_train[skip:])**2)**0.5 / th_train.std() * 100, '%')

# OE simulation on test data
th_test_sim = OE_Simulation(u_test, fmodel_OE, 50, skip=skip)

assert len(th_test_sim) == len(th_test)
np.savez('../gym-unbalanced-disk/disc-benchmark-files/hidden-test-simulation-example-submission-file.npz', th=th_test_sim, u=u_test)

train simulation errors:
RMS: 0.5817430545176709 radians
RMS: 33.331421784911505 degrees
NRMS: 121.36402278413567 %
